# Rote Learning

In [1]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Der α-β-Pruning Algorithmus versucht durch die Auswertung der zukünftig möglichen Züge den Besten auszuwählen. Da der Suchraum jedoch zu groß ist um jedes mögliche Spiel zu betrachten, muss an einem bestimmten Punkt das Vorausschauen beendet werden und eine Schätzung statt dem tatsächlichen Wert verwendet werden. Diese Aufgabe übernimmt die in dem Notebook `nmm-heuristic` implementierte Heuristik.
Dies bedeutet dass eine künstliche Intelligenz, die den α-β-Pruning Algorithmus implementiert, je besser ist, desto mehr Schritte in die die Zukunft geschaut werden kann. Die Laufzeit α-β-Pruning Algorithmus nimmt jedoch mit zunehmender Tiefe, aufgrund der sehr schnell ansteigenden Anzahl der Zustände, deutlich zu. Somit ist eine Neuberechnung der Schäzung bei jedem Zug nicht praktikabel.

An diesem Punkt setzt das Rote-Learning an: Indem der α-β-Pruning Algorithmus um eine elementare Form des Lernens erweitert wird, wird dessen Effektivität stark gesteigert. Grundlegend werden bei der Verwendung von Rote-Learning alle Zustände, die jemals besucht wurden, zusammen mit den dazugehörigen errechneten Schätzungen abgespeichert. Anstatt die Wertschätzung dieser Zustände bei jedem Zug neu zu berechnen, können diese nun aus dem Speicher abgerufen werden. Dies spart vor allem bei einer hohen Suchtiefe viel Rechenzeit ein. Diese Einsparung der Rechenzeit kann darauf verwendet werden, weitere Zustände zu berechnen und somit die Qualität der Schätzungen zu erhöhen. Da mit jedem Spiel mehr Zustände und deren Schätzungen gespeichert werden, verbessert sich das Ergebnis des Algorithmus über Zeit. Es tritt also ein Lerneffekt ein.

## Implementierung

Um das oben genannte Prinzip von Rote-Learning in Python zu implementieren, wurde auf eine bereits fertige Implementierung des Alpha-Beta-Prunings Algorithmus für das Spiel Mühle zurück gegriffen. Dieser wurde im Rahmen der Studienarbeit entwickelt und musste für die Verwendung von Rote-Learning nur noch geringfügig angepasst werden. Um zu evaluieren ob Rote-Learning einen Vorteil gegenüber einem nicht trainierten Algorithmus herbeiführt, werden folgende Notebooks benötigt:

- `nmm-cache` implementiert eine schnelle, persistierbare Transpositionstabelle (auch Cache genannt);
- `nmm-rote-training` implementierung ein Klasse für den Trainingsteil des Rote-Learnings;
- `nmm-alpha-beta-pruning` ist eine bereits vorhandene Implementierung des α-β-Pruning Algorithmus;
- `nmm-tournament` ist eine ebenfalls bereits vorhandene Implementierung zur Evaluation, ob ein per Rote-Learning trainierter Algorithmus besser abschneidet als ein nicht trainierter Algorithmus.

In [2]:
%run ./nmm-cache.ipynb
%run ./nmm-cache-rote-learning.ipynb
%run ./nmm-rote-training.ipynb
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-tournament.ipynb
%run ./nmm-alpha-beta-pruning-rote-learning.ipynb

Desweitern wird das Paket `memory_profiler` eingebunden, welches die Überwachung der Auslastung des Arbeitsspeichers ermöglicht.

In [3]:
from memory_profiler import memory_usage

### Cache

Um den bei Rote-Learning beschriebenen Lerneffekt zu erzielen und die Zustände zu Speichern, wurde eine Transpositionstabelle als Klasse `Cache` in dem Notebook `nmm-cache` implementiert. Diese Klasse verfügt über fünf Methoden zur Verwaltung des Caches:

- `write` speichert einen neuen Zustand und die errechneten Werte ab;
- `read` liest die gespeicherten Werte eines Zustandes aus dem Cache aus;
- `save` legt den Inhalt des Caches in einer Datei auf einem Datenträger ab;
- `load` lädt die Daten des Caches aus einer Datei auf dem Datenträger;
- `clean` löscht Einträge aus dem Cache, falls die Anzahl der Einträge `max_size` überschritten wurde.

Um die einzelnen Werte für die Zustände im Cache abzulegen, werden sowohl die Zustände als auch die Werte in ein Byte-Array konvertiert. Insgesamt ist ein Eintrag des Caches somit 32 Bytes groß. Um zu verhindern, dass der Cache zu groß wird um ihn im Arbeitsspeicher oder auf dem Datenträger speichern zu können, wurde der Parameter `max_size` eingeführt. Dieser begrenzt den Cache auf eine maximale Anzahl an Einträgen. Um die Performance zu steigern, wird dies jedoch nicht beim jedem Aufruf der `write` Methode geprüft sondern nur beim Aufruf der Methode `clean`. Dies kann dazu führen, dass zwischen den Aufrufen der `clean` Methode mehr Einträge im Cache vorhanden sind, als durch `max_size` erlaubt.
Wird der Cache auf dem Datenträger abgelegt, entspricht die Dateigröße genau der Anzahl an Elementen multipliziert mit 32 Bytes. Die Größe das Caches im Arbeitsspeicher ist jedoch um einen Faktoren von ca. $6$ größer, da Python weiteren Arbeitsspeicher, beispielsweise für den Datentyp, reserviert.

Der folgende Befehl erstellt einen Cache mit einer Größe von 50 Millionen Zuständen.

In [4]:
cache = Cache(max_size = 50_000_000)

### Anbindung an α-β-Pruning

Da im α-β-Pruning Algorithmus bereits die Verwendung eines In-Memory-Caches vorgesehen ist, mussten nur kleine Anpassungen vorgenommen werden. Es musste sichergestellt werden, dass sowohl zum Schreiben als auch zum Lesen der Werte aus dem Cache die richtigen Methoden aufgerufen werden, sowie dass ein Cache entweder als Parameter übergeben werden kann oder ein neuer instanziiert wird. Nach jeder Berechnung der nächsten besten Züge mit Hilfe der `bestMoves` Methode, wurde der Aufruf der `clean` Methode hinterlegt, um die Größe des Caches anzupassen.

###  Traingingsprozess

Um die Transpositionstabelle zu trainieren und somit das Rote-Learning umzusetzen, wird die Klasse `Training` aus dem Notebook `nmm-rote-training` verwendet. Dazu muss zuerst eine Methode erstellt werden, welche den Algorithmus für die künstliche Intelligenz konfiguriert und generiert. Diese Methode wird `artificial_intelligence_generator` genannt und generiert eine AlphaBetaPruning Instanz, welche den übergebenen Cache und eine benutzerdefinierte Heuristiken verwendet. Die verwendeten Heuristiken wurden bereits im Rahmen der Studienarbeit ermittelt.

In [5]:
def artificial_intelligence_generator_primitiv(cache: Cache):
    customWeights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1)
    return AlphaBetaPruning(cache = cache, weights = customWeights)

In [6]:
def artificial_intelligence_generator_rote_learning(cache: Cache):
    customWeights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1)
    return AlphaBetaPruning(cache = cache, weights = customWeights)

Für den Cache wurde sich für eine maximale Größe von 50.000.000 Einträgen entschieden. Dies resultiert in eine maximal Größe des Caches von ca. 1,6Gb auf dem Datenträger. Des Weiteren werden die Standardwerte der Training-Klasse verwendet. Das heißt, dass ingesamt 100 Spiele gespielt werden und der Seed für den Zufallsgenerator nicht angepasst wird. Alle 10 Spiele wird der Cache auf dem Datenträger gespeichert und wird der Prefix `training-` für den Dateinamen verwendet.

In [7]:
training = Training(
    cache = cache, 
    artificial_intelligence = artificial_intelligence_generator_rote_learning,
    path_prefix = 'training-rote-small-'
)

Der Trainingsprozess wird durch den Aufruf der Funktion `train` gestartet und dauert mit der oben genannten Konfiguration in etwa 4,5 Stunden. Dabei werden bis zu 10 Gb Arbeitsspeicher benötigt.

In [ ]:
mem_usage = memory_usage(training.train)
print('Maximum memory usage: %s MB.' % max(mem_usage))

Training #training-rote-small-001:


 > Cache(size=17, max_size=50000000)

Training #training-rote-small-002:


 > Cache(size=33, max_size=50000000)

Training #training-rote-small-003:


 > Cache(size=49, max_size=50000000)

Training #training-rote-small-004:





 > Cache(size=65, max_size=50000000)

Training #training-rote-small-005:


...

 > Cache(size=299, max_size=50000000)

Training #training-rote-small-020:


 > Cache(size=315, max_size=50000000)
> Saving to 'training-rote-small-020.cache'...




Training #training-rote-small-021:


 > Cache(size=331, max_size=50000000)

Training #training-rote-small-022:


 > Cache(size=346, max_size=50000000)

Training #training-rote-small-023:


### Trainings Limitierungen

Auf den ersten Blick scheint es so, dass man den Cache unendlich weiter trainieren kann. Dies jedoch mit der aktuellen Implementierung nicht zielführend. Ab einem gewissen Trainingszeitpunkt ist das Rekursionslimit der Einträge im Cache so hoch, dass dieses durch weiteres Training nicht erneut erreicht werden können. Dies ist lässt sich gut an nachfolgendem Beispiel erkennen:

Durch die geringe Größe des Caches sowie dem kleinen Wert für `max_states` kommt man sehr schnell an den Punkt, an dem das Cache-Limit auf 3 erhöht wird. Dadurch werden fast alle Einträge aus dem Cache gelöscht und es verbleiben nur noch `48` im Cache. Dies passiert mehrere Male und ist ein Zeichen dafür, dass der Cache nicht mehr weiter trainiert werden kann, da dieser zu klein ist um genügend Einträge zu halten, damit mehr Einträge mit `limit = 3` berechnet werden können bevor der `max_states` Wert überschritten wird.

In [ ]:
t = Training(
    cache                   = Cache(max_size = 1_000),
    artificial_intelligence = lambda cache: AlphaBetaPruning(
        cache      = cache,
        max_states = 1_000
    ),
    path_prefix             = "max-learning-",
    save_interval           = 1
)
#t.train()

## Auswertung

Bei der Auswertung wird auf die Klasse `Tournament` aus dem Notebook `nmm-tournament` zurück gegriffen. Diese Klasse besitzt die Methode `play`, welche es ermöglicht verschiedene Algorithmen gegeneinander Antreten zu lassen und die Ergebnisse zu speichern.

Um zu ermitteln ob die Rote-Learning-Methode einen Vorteil gegenüber einem normalen Cache bietet, tritt eine über 100 Spiele trainierte Trainspositionstabelle gegen eine untrainierte Transpositionstabelle an. Dies wird zehn mal mit verschiedenen Seeds wiederholt. Das verändern der Seeds sorgt dabei für einen veränderten Spielverlauf.
Somit sollte ermittelt werden können, ob durch das Training eine Verbesserung der künstlichen Intelligenz eintritt und wenn ja, wie deutlich diese Verbesserung ausfällt.

In [ ]:
for i in range(10):
    Tournament(
        [
            lambda: AlphaBetaPruning(
                name='Primitiv',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000, path = 'training-small-final.cache')
            ),
            lambda: AlphaBetaPruning(
                name='Rote Learning',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000, path = 'training-rote-small-final.cache')
            )
        ],
        instances_per_round = 1,
        seed_offset         = i,
        name                = f"small-full-{i}-seed"
    ).play()

Die Auswertung hat ergeben, dass von insgesamt von 20 gespielten Spielen 10 gewonnen wurden. Bei 5 lief es auf ein Unentschieden herraus und 5 wurden verloren. Damit lässt sich mit ziemlicher Sicherheit sagen, dass das Training der ersten Spielphase durch Rote-Learning einen Vorteil im gesamten Spielverlauf bietet.

## Fazit
Das Ziel dieser Arbeit war die Implementierung des in dem Paper `Some Studies in Machine Learning Using the Game of Checkers` von `A.L.Samuel` beschriebene Prinzip von Rote-Learning zu erarbeiten. Das Ziel wurde erfolgreich durch eine Python-Implementierung erreicht und durch eine Verbesserung der Leistung einer künstlichen Intelligenz bestätigt. Im Laufe der Arbeit wurde sich neben der Implementierung in Python auch mit den dem α-β-Pruning Algorithmus sowie verschiedenen Methoden zur Persistierung eines Caches beschäftigt.

Während in dem Paper jedoch zwei Methoden beschrieben werden, um Einträge aus einem zu großen Cache zu löschen, wird sich in dieser Arbeit nur auf eine Methode konzentriert. Im Paper wird zum einen die Aufruf-Frequenz oder zum anderen das Rekursionslimit des Eintrags. Diese Arbeit behandelt nur das Löschen von Einträgen basierend auf dem Rekursionslimit.

Dennoch lässt sich sagen, dass diese Arbeit ein Erfolg war, da eine deutliche Verbesserung des α-β-Pruning mit einem trainierten Cache erreicht werden konnte, sodass nach  100 Trainingsrunden nur noch 25% der Spiele gegen eine nicht trainierte künstliche Intelligenz verloren wurden.